In [17]:
from pathlib import Path
import pandas as pd
import numpy as np
import stockstats

In [18]:
file_path = Path('Resources/stock_data_csv.csv')
df=pd.read_csv(file_path,index_col="date_time")

In [19]:
gme=df[["gme_px","gme_vol","spy_px"]]
gme=pd.DataFrame(gme)
gme.rename(columns={"gme_px":"close","gme_vol":"vol","spy_px":"spy"},inplace=True)

In [20]:
amc=df[["amc_px","amc_vol","spy_px"]]
amc=pd.DataFrame(amc)
amc.rename(columns={"amc_px":"close","amc_vol":"vol","spy_px":"spy"},inplace=True)

In [21]:
bb=df[["bb_px","bb_vol","spy_px"]]
bb=pd.DataFrame(bb)
bb.rename(columns={"bb_px":"close","bb_vol":"vol","spy_px":"spy"},inplace=True)

In [22]:
bbby=df[["bbby_px","bbby_vol","spy_px"]]
bbby=pd.DataFrame(bbby)
bbby.rename(columns={"bbby_px":"close","bbby_vol":"vol","spy_px":"spy"},inplace=True)

In [23]:
nok=df[["nok_px","nok_vol","spy_px"]]
nok=pd.DataFrame(nok)
nok.rename(columns={"nok_px":"close","nok_vol":"vol","spy_px":"spy"},inplace=True)

In [24]:
def WR(stock,wr_period):
    stock["max"]=stock["close"].rolling(window=wr_period).max()
    stock["min"]=stock["close"].rolling(window=wr_period).min()
    stock["shift"]=stock["close"].shift(1)
    stock["WR"]=((stock["shift"]-stock["max"])/(stock["max"]-stock["min"]))*100
    stock.drop(columns=["max","min","shift"],inplace=True)
    return stock


In [25]:
def RSI(stock,rsi_period):
    chg = stock["close"].diff(1)
    stock["gain"] = chg.mask(chg<0,0)
    stock["loss"] = chg.mask(chg>0,0)
    stock["avg_gain"] = stock["gain"].rolling(window=rsi_period).mean()
    stock["avg_loss"] = stock["loss"].rolling(window=rsi_period).mean()

    stock["rs"]=abs(stock["avg_gain"]/stock["avg_loss"])
    stock["RSI"] = 100-(100/(1+stock["rs"]))
    stock.drop(columns=["gain","loss","avg_gain","avg_loss","rs"],inplace=True)
    return stock

In [26]:
def Beta(stock,beta_period):
    stock["return"]=stock["close"].pct_change()
    stock["spy_return"]=stock["spy"].pct_change()    
    stock["cov"]=stock.rolling(window=beta_period).cov().unstack()["return"]["spy_return"]
    stock["var"]=stock["spy_return"].rolling(window=beta_period).var()
    stock["Beta"]=stock["cov"]/stock["var"]
    stock.drop(columns=["spy_return","cov","var"],inplace=True)
    return stock

In [27]:
def OBV(stock):
    stock["chg"]=stock["close"].diff(1)
    stock['up-down']= np.where(stock['chg']< 0, -1.0, 1.0)
    stock["po-ne"]=stock["vol"]*stock["up-down"]
    stock["OBV"]=stock["po-ne"].cumsum()
    stock.drop(columns=["chg","up-down","po-ne"],inplace=True)
    return stock

In [35]:
Beta(gme,180)
WR(gme,14)
RSI(gme,14)
OBV(gme)
gme.to_csv('Results/GME.csv') 

In [36]:
Beta(amc,180)
WR(amc,14)
RSI(amc,14)
OBV(amc)
amc.to_csv('Results/AMC.csv') 

In [37]:
Beta(bb,180)
WR(bb,14)
RSI(bb,14)
OBV(bb)
bb.to_csv('Results/BB.csv') 

In [38]:
Beta(bbby,180)
WR(bbby,14)
RSI(bbby,14)
OBV(bbby)
bbby.to_csv('Results/BBBY.csv') 

In [39]:
Beta(nok,180)
WR(nok,14)
RSI(nok,14)
OBV(nok)
nok.to_csv('Results/NOK.csv') 